# Web Demo

> Interactive web demo for SQLite MCP Server

In [ ]:
#| default_exp web_demo

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import FileResponse
from pydantic import BaseModel
import sqlite3
import os
from typing import List, Tuple, Dict, Any
from llamaindex_mcp_nbdev.core import init_db

In [ ]:
#| export

# Create FastAPI app
app = FastAPI(title="SQLite MCP Server Demo")

# Add CORS middleware to allow browser requests
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # In production, specify your domain
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Request models
class QueryRequest(BaseModel):
    query: str

# Database path
DB_PATH = "demo.db"

In [ ]:
#| export

# Initialize database on startup
@app.on_event("startup")
async def startup_event():
    """Initialize database on server startup"""
    init_db(DB_PATH)
    print(f"Database initialized at: {DB_PATH}")

In [ ]:
#| export

@app.post("/add_data")
async def add_data(request: QueryRequest):
    """Add data to the database"""
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        
        # Validate that it's an INSERT query
        if not request.query.strip().upper().startswith("INSERT"):
            raise HTTPException(status_code=400, detail="Only INSERT queries are allowed")
        
        cursor.execute(request.query)
        conn.commit()
        conn.close()
        
        return {"success": True, "message": "Data added successfully"}
    
    except sqlite3.Error as e:
        return {"success": False, "error": str(e)}
    except Exception as e:
        return {"success": False, "error": str(e)}

In [ ]:
#| export

@app.post("/read_data")
async def read_data(request: QueryRequest):
    """Read data from the database"""
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        
        # Validate that it's a SELECT query
        if not request.query.strip().upper().startswith("SELECT"):
            raise HTTPException(status_code=400, detail="Only SELECT queries are allowed")
        
        cursor.execute(request.query)
        results = cursor.fetchall()
        conn.close()
        
        return {"success": True, "data": results}
    
    except sqlite3.Error as e:
        return {"success": False, "error": str(e)}
    except Exception as e:
        return {"success": False, "error": str(e)}

In [ ]:
#| export

@app.get("/")
async def serve_demo():
    """Serve the demo HTML page"""
    html_path = os.path.join(os.path.dirname(os.path.dirname(__file__)), "demo.html")
    if os.path.exists(html_path):
        return FileResponse(html_path)
    else:
        return {"message": "Demo page not found. Please ensure demo.html is in the project root."}

In [ ]:
#| export

@app.get("/stats")
async def get_stats():
    """Get database statistics"""
    try:
        conn = sqlite3.connect(DB_PATH)
        cursor = conn.cursor()
        
        # Get total count
        cursor.execute("SELECT COUNT(*) FROM people")
        total = cursor.fetchone()[0]
        
        # Get average age
        cursor.execute("SELECT AVG(age) FROM people")
        avg_age = cursor.fetchone()[0] or 0
        
        # Get professions count
        cursor.execute("SELECT profession, COUNT(*) FROM people GROUP BY profession")
        professions = cursor.fetchall()
        
        conn.close()
        
        return {
            "success": True,
            "total_records": total,
            "average_age": round(avg_age, 2) if avg_age else 0,
            "professions": {prof: count for prof, count in professions}
        }
    
    except Exception as e:
        return {"success": False, "error": str(e)}

In [ ]:
#| export

def run_demo():
    """Run the web demo server"""
    import uvicorn
    print("Starting web demo server...")
    print("Open http://localhost:8000 in your browser to view the demo")
    uvicorn.run(app, host="0.0.0.0", port=8000)

In [ ]:
#| export
#| eval: false

if __name__ == "__main__":
    run_demo()

## Testing the API endpoints

In [ ]:
# Test database initialization
test_conn, test_cursor = init_db('test_web.db')
test_cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='people'")
assert test_cursor.fetchone() is not None, "People table should exist"
test_conn.close()
print("Database initialization test passed!")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()